In [9]:
import requests
import time
import random
from fake_useragent import UserAgent
ua = UserAgent()

def fetch_specific_pages(stock_code, start_page=500, end_page=1000):
    all_htmls = []
    for page in range(start_page, end_page + 1):
        try:
            if page == 1:
                url = f'https://guba.eastmoney.com/list,{stock_code}.html'
            else:
                url = f'https://guba.eastmoney.com/list,{stock_code}_{page}.html'
            
            headers = {
                 "user-agent": ua.random,
                'Referer': f'https://guba.eastmoney.com/list,{stock_code}.html'
            }
            
            # 随机延迟，避免触发反爬
            time.sleep(random.uniform(1.5, 3.5))
            
            response = requests.get(url, headers=headers, params={'type': 1}, timeout=10)
            response.encoding = 'utf-8' 
            
            if response.status_code == 200:
                all_htmls.append(response.text)
                print(f"成功获取第 {page} 页数据")
                print(response.text[:1000]) 
                print("...\n")  
            else:
                print(f"第 {page} 页请求失败，状态码：{response.status_code}")
                
        except Exception as e:
            print(f"第 {page} 页抓取异常：{str(e)}")
    
    return all_htmls

stock_code = "zssh000001,99"       #zssh000001
start_page = 737                    
end_page = 740

all_html_content = fetch_specific_pages(stock_code, start_page, end_page)
print(f"\n共 {len(all_html_content)} 页数据成功抓取")


成功获取第 737 页数据
<!DOCTYPE html><html lang="en"><meta charSet="UTF-8"/><head><meta name="env" content="zp, render at 2025-04-08 15:34:52"/><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"/><meta name="renderer" content="webkit"/><meta charSet="UTF-8"/><link rel="icon" href="//gbfek.dfcfw.com/project/guba2022/img/favicon.ico"/><title>上证指数(000001)股吧_上证指数怎么样_分析讨论社区—东方财富网</title><meta name="keywords" content="上证指数股吧,000001股吧,上证指数怎么样,上证指数分析"/><meta name="description" content="上证指数(000001)股吧，股民朋友可以在这里畅所欲言，分析讨论股票名的最新动态。东方财富股吧，专业的股票论坛社区。"/><meta name="mobile-agent" content=""/><link rel="canonical" href=""/><base target="_blank"/><script>window.shimingOption = 2;</script><script src="//gbfek.dfcfw.com/project/guba/modules/h5Adaptation.js?r=6"></script><img id="weixin-share" src="//cmsjs.eastmoney.com/common/weixin-share.png" style="position:absolute;height:0px;width:0px;left:-10000px"/></head><base target="_blank"/><link rel="stylesheet" href="//gbfek.dfcfw.com/deploy/fd_guba_web2022

In [10]:
from bs4 import BeautifulSoup

def parse_urls(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    tbody = soup.find('tbody', class_='listbody')
    if not tbody:
        return []
    
    post_urls = []
    for tr in tbody.find_all('tr', class_='listitem'):
        try:
           
            title_anchor = tr.find('div', class_='title').find('a')
            if not title_anchor:
                continue
                
            href = title_anchor.get('href', '')
            
       
            if href.startswith('//'):
                href = f'https:{href}'
            elif href.startswith('/'):
                href = f'https://guba.eastmoney.com{href}'
            
            post_urls.append(href)
        except Exception as e:
            print(f"解析异常：{str(e)}")
            continue

    return post_urls


post_urls = []
for idx, html in enumerate(all_html_content, 1):
    page_urls = parse_urls(html)
    post_urls.extend(page_urls)
    print(f"第{idx}页解析完成，获得{len(page_urls)}条URL")


print("\n" + "="*50)
print(f"共获取 {len(post_urls)} 条帖子URL")
print("="*50 + "\n")


print("URL示例：")
for i, url in enumerate(post_urls[:10], 1):
    print(f"[{i}] {url}")

# 保存全部URL到文件
# with open('post_urls.txt', 'w') as f:
#     f.write('\n'.join(post_urls))

第1页解析完成，获得80条URL
第2页解析完成，获得80条URL
第3页解析完成，获得80条URL
第4页解析完成，获得80条URL

共获取 320 条帖子URL

URL示例：
[1] https://caifuhao.eastmoney.com/news/20240126142113628165440
[2] https://caifuhao.eastmoney.com/news/20240126134741684622110
[3] https://caifuhao.eastmoney.com/news/20240126133751939663690
[4] https://caifuhao.eastmoney.com/news/20240126131843192931270
[5] https://caifuhao.eastmoney.com/news/20240126125939688172050
[6] https://caifuhao.eastmoney.com/news/20240126123914878618190
[7] https://caifuhao.eastmoney.com/news/20240126112112282122090
[8] https://caifuhao.eastmoney.com/news/20240126105137133314560
[9] https://guba.eastmoney.com/news,zssh000001,1394942458.html
[10] https://caifuhao.eastmoney.com/news/20240125211823361463760


In [ ]:
 import requests
from bs4 import BeautifulSoup
import json
import random
import time
from fake_useragent import UserAgent  # 导入 fake-useragent


ua = UserAgent()


headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "zh-CN,zh;q=0.9,en;q=0.8",
    "cache-control": "max-age=0",
    "connection": "keep-alive",
    "user-agent": ua.random, 
}


results = []  

for url in post_urls:
    try:
        resp = requests.get(url, headers=headers, timeout=10)
        resp.raise_for_status()
        resp.encoding = "utf-8"
        
        soup = BeautifulSoup(resp.text, "html.parser")
        content_div = soup.find("div", class_="article")
        content = content_div.get_text(strip=True) if content_div else "未能找到正文内容"
        

        json_data = {"input": content}
        print(json.dumps(json_data, ensure_ascii=False, indent=4))
        

        results.append(json_data) 
    
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP Error occurred for {url}: {http_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"Request Error occurred for {url}: {req_err}")
    
    print("\n" + "-" * 80 + "\n")
    time.sleep(15)


{
    "input": "洗洗更健康！财经点将录2024年01月26日 14:21广东返回\r\n            上证指数吧>点赞0评论0收藏大中小from：炒股不慌A股在连涨几天，累了，需要休息下，今天也如预期走出小幅震荡的行情，甚至还在接近午盘时，出现一定程度的回落，整体上没有太大的问题，洗洗休息下更健康。现在权重已经搭好台，接下来指数只会震荡上涨，也该轮到题材唱戏的时候。市场情绪在连续强势后，今天也出现分化，也是在预期中的。这轮共振起来的3板，13家早上只掉队了2家，晋级率依然很高，下午还得继续分化。接下来大概率从中间重两头轻逐步演绎成连板高标之间的搏杀，特别要注意的是下周一这个时间节点，因为4板到时候会晋级到5板，5板难度就加大，所以下周这一节点需要多注意~！追加内容本文作者可以追加内容哦 !举报"
}

--------------------------------------------------------------------------------

{
    "input": "买进301113专心的投资人2024年01月26日 13:47湖北返回\r\n            上证指数吧>点赞0评论0收藏大中小户外休闲家具业 以中国为代表的发展中国家市场潜力巨大,将为本行业提供广阔的发展空间。目前,我国已发展成为全球户外休闲家具及用品的制造中心,产业配套完整。我国户外休闲家具行业的优势企业凭借其不断提升的设计能力和生产技术水平,国际竞争力不断加强。户外休闲家具需求呈多样化发展趋势,具体体现在:市场对个性化、高端化产品的需求日益增加,同时因文化、消费偏好、气候环境的不同造成对产品的需求日益多样化。产品开发与设计能力是决定企业产品附加值、技术含量以及品牌竞争力的重要因素。户外休闲家具制造商需要快速跟踪不同市场需求变化,加强产品开发与设计能力建设,推出适合客户需求的个性化户外休闲家具及用品定制方案,以满足客户对产品的个性化需求。未来随着消费者消费观念的升级,品牌户外休闲家具企业的自主设计研发水平将直接主导其产品的溢价能力。追加内容本文作者可以追加内容哦 !举报"
}

--------------------------------------------------------------------------------


In [12]:
import json
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")
filename = f"第{start_page}页到{end_page}_{timestamp}.json" 


with open(filename, 'w', encoding='utf-8') as f:
    
    json.dump(results, f, ensure_ascii=False, indent=2)
    
print(f"数据已保存到 {filename}，共 {len(results)} 条记录")

✅ 数据已保存到 第737页到740_2025-04-08_17-37.json，共 319 条记录
